In [1]:
import pandas as pd
from datetime import datetime
import xlrd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

China: datos faltantes antes de 02/01/2006

In [2]:
cotizaciones = pd.read_excel('../data/cotizaciones 1997.xlsx', sheet_name=0)
cotizaciones_usd = pd.read_excel('../data/cotizaciones 1997.xlsx', sheet_name=1)
lista_paises = cotizaciones_usd.columns[1:-3]

var_cotizaciones_usd = cotizaciones_usd.copy()
var_cotizaciones_usd=var_cotizaciones_usd.groupby(['anio','mes'],as_index=False).mean()
var_cotizaciones_usd['Período']=pd.to_datetime('1/'+var_cotizaciones_usd.mes.astype(str)+'/'+var_cotizaciones_usd.anio.astype(str), format='%d/%m/%Y')
# var_cotizaciones_usd=var_cotizaciones_usd.fillna(method='bfill')
for i in range(len(lista_paises)):
    var_cotizaciones_usd[lista_paises[i]
                         ] = var_cotizaciones_usd[lista_paises[i]].pct_change(12)

var_cotizaciones_usd.drop(['mes','anio'],axis=1,inplace=True)
cotizaciones_usd.drop(['dia','mes','anio'],axis=1,inplace=True)

var_cotizaciones_usd

,Brasil,Canadá,Chile,Argentina,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam,dia,Período
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.5,1997-11-01
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1997-12-01
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1998-01-01
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,1998-02-01
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.0,1998-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,-0.000201,0.046780,0.185366,0.287763,-0.002271,-0.087038,0.041943,0.061329,0.213753,0.135958,0.066868,0.137771,0.009177,15.5,2022-06-01
296,0.038633,0.033117,0.263720,0.334644,0.027530,-0.062577,0.039471,0.067078,0.237190,0.150238,0.055457,0.160289,0.015889,16.0,2022-07-01
297,-0.021361,0.025148,0.158636,0.391966,0.001418,-0.064497,0.049537,0.073300,0.232497,0.152667,0.046792,0.162669,0.024046,16.0,2022-08-01
298,-0.008130,0.052017,0.178988,0.456643,0.003015,-0.041754,0.085520,0.090031,0.298236,0.214553,0.055411,0.189250,0.037399,15.5,2022-09-01


In [3]:
def plot_tasa_deva(df=var_cotizaciones_usd, anio='2007', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio)}-{str(mes)}-{str(dia)}'].index[0]
    var_cotizaciones_usd = df[df.index >= idx].reset_index(drop=True)

    x = var_cotizaciones_usd["Período"]
    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Real", x=x, y=var_cotizaciones_usd["Brasil"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Yuan", x=x, y=var_cotizaciones_usd["China"], mode="lines"))
    # cotizaciones_plot.add_trace(go.Scatter(name = "ARS-USD", x=x, y=var_cotizaciones_usd["Argentina"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Euro", x=x, y=var_cotizaciones_usd["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación", tickformat= ',.0%')
    cotizaciones_plot.update_yaxes(
        title_text="Tasa de devaluación", tickformat=',.0%')
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    height=600, width=1000,
                                    template='none',
                                    title_text=f"Tasa de devaluación de las monedas de los tres principales socios comerciales <br><sup>Variación interanual con frecuencia mensual",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


def evolucion_monedas(df=cotizaciones_usd, anio='2006', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio)}-{str(mes)}-{str(dia)}'].index[0]
    cotizaciones_usd = df[df.index >= idx].reset_index(drop=True)
    x = cotizaciones_usd["Período"]

    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(name=f"Brasil", x=x, y=cotizaciones_usd["Brasil"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(name=f"China", x=x, y=cotizaciones_usd["China"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(name=f"Euro", x=x, y=cotizaciones_usd["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    height=600, width=1000,
                                    template='none',
                                    title_text=f"Evolución de las paridades con el USD de los tres principales socios comerciales <br><sup>Frecuencia diaria",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


def indice_monedas(df=cotizaciones_usd, anio='2006', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio)}-{str(mes)}-{str(dia)}'].index[0]
    df = df[df.index >= idx].reset_index(drop=True)
    lista_paises=df.columns[1:]
    var_cotizaciones_usd = df.copy()
    base = var_cotizaciones_usd[var_cotizaciones_usd.Período == '2019-01-01'].index[0]
    for i in range(len(lista_paises)):
        var_cotizaciones_usd[lista_paises[i]] = var_cotizaciones_usd[lista_paises[i]
                                                                     ]/var_cotizaciones_usd[lista_paises[i]].iloc[base]*100
    # var_cotizaciones_usd.dropna(inplace=True)

    x = var_cotizaciones_usd["Período"]

    # Datos
    cotizaciones_plot = go.Figure()
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Real", x=x, y=var_cotizaciones_usd["Brasil"], mode="lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Yuan", x=x, y=var_cotizaciones_usd["China"], mode="lines"))
    # cotizaciones_plot.add_trace(go.Scatter(name = "ARS-USD", x=x, y=var_cotizaciones_usd["Argentina"], mode = "lines"))
    cotizaciones_plot.add_trace(go.Scatter(
        name="USD-Euro", x=x, y=var_cotizaciones_usd["Zona Euro"], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    # cotizaciones_plot.update_yaxes(title_text="Tasa de devaluación", tickformat= ',.0%')
    cotizaciones_plot.update_yaxes(
        title_text="Base enero 2019=100")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    margin ={'b': 50,'l':50,'r':15},
                                    height=600, width=900,
                                    template='none',
                                    title_text=f"Evolución de las monedas de los tres principales socios comerciales <br><sup>índice simple con base enero 2019=100",
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.3, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.09,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


# plot_tasa_deva(anio=1999)

evolucion_monedas(anio=2021,mes=1)

indice_monedas(anio=2007)


In [4]:
cotizaciones_usd=cotizaciones_usd.fillna(method='bfill')
cotizaciones=cotizaciones.fillna(method='bfill')

In [5]:
itcrm = pd.read_excel('../data/ITCRMSerie.xlsx', header=1, skipfooter=4)

ponderadores = pd.read_excel('../data/ITCRMSerie.xlsx', sheet_name=2, header=1)
ponderadores.Período = pd.to_datetime(
    ponderadores['Período'], format='%d/%m/%Y')


def evolucion_ponderaciones():
    x = ponderadores["Período"]

    # Datos
    cotizaciones_plot = go.Figure()
    for index, pais in enumerate(ponderadores.columns[1:]):
        cotizaciones_plot.add_trace(go.Scatter(
            name=f"{pais}", x=x, y=ponderadores[pais], mode="lines"))

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(title_text="Ponderaciones %")
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    height=600, width=1000,
                                    template='none',
                                    title_text=f"Evolución de las ponderaciones publicadas por el BCRA <br><sup>Frecuencia mensual",
                                    legend=dict(yanchor="top", y=-.05, xanchor="left", x=0.36, orientation='h'))

    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)
    return cotizaciones_plot


evolucion_ponderaciones()


In [6]:
# Tipo de cambio real

fin = itcrm.Período.iloc[-1].strftime('%m/%d/%Y')
inicio = itcrm.Período[0].strftime('%m/%d/%Y')
ponderadores.Período = ponderadores.Período.apply(
    lambda x: x.strftime('%d/%m/%Y'))
itcrm.Período = itcrm.Período.apply(lambda x: x.strftime('%d/%m/%Y'))

monthDates = pd.DataFrame({
    'Período': pd.date_range(start=inicio, end=fin, freq='d').strftime('%d/%m/%Y')
})

ponderaciones_completo = (monthDates[['Período']].merge(ponderadores, on='Período', how='left').fillna(method='ffill')
                          .merge(itcrm[['Período']], on='Período', how='right'))

# ponderaciones_completo[ponderaciones_completo.Período=='01/01/2004']
ponderaciones_completo


,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam
0,01/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
1,02/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
2,03/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
3,04/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
4,05/01/1997,29.562711,0.797194,5.098276,20.522159,2.301729,3.338386,2.323775,0.854147,2.321107,2.448984,0.795496,29.620689,0.015347
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,15/10/2022,25.316588,1.054015,4.454715,12.635233,2.421166,2.285634,19.194241,7.041181,1.577486,1.107633,1.818445,18.100860,2.992803
9419,16/10/2022,25.316588,1.054015,4.454715,12.635233,2.421166,2.285634,19.194241,7.041181,1.577486,1.107633,1.818445,18.100860,2.992803
9420,17/10/2022,25.316588,1.054015,4.454715,12.635233,2.421166,2.285634,19.194241,7.041181,1.577486,1.107633,1.818445,18.100860,2.992803
9421,18/10/2022,25.316588,1.054015,4.454715,12.635233,2.421166,2.285634,19.194241,7.041181,1.577486,1.107633,1.818445,18.100860,2.992803


In [7]:
cotizaciones_final = (ponderaciones_completo[['Período']].merge(cotizaciones, on='Período', how='left')
                     .fillna(method='ffill')
                     .drop_duplicates('Período', ignore_index=True)
                     )

indice = cotizaciones_final[cotizaciones_final.Período ==
                           cotizaciones.Período.iloc[0]].index
cotizaciones_final = cotizaciones_final[cotizaciones_final.index >= indice[0]].reset_index(
    drop=True)

cotizaciones_final


,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam
0,21/11/1997,0.904404,0.706700,0.002366,1.00,0.121803,0.100604,0.375548,0.139803,0.007959,1.691000,0.706400,1.181000,0.001639
1,22/11/1997,0.904404,0.706700,0.002366,1.00,0.121803,0.100604,0.375548,0.139803,0.007959,1.691000,0.706400,1.181000,0.001639
2,23/11/1997,0.904404,0.706700,0.002366,1.00,0.121803,0.100604,0.375548,0.139803,0.007959,1.691000,0.706400,1.181000,0.001639
3,24/11/1997,0.902039,0.704700,0.002332,1.00,0.121507,0.101214,0.375548,0.139803,0.007888,1.692500,0.713100,1.181000,0.001639
4,25/11/1997,0.902039,0.706200,0.002332,1.00,0.121507,0.101214,0.375548,0.139803,0.007851,1.685500,0.711800,1.181000,0.001639
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,15/10/2022,28.503759,109.258592,0.157873,151.64,7.553109,3.681029,21.083073,1.841185,1.019155,169.472864,150.780551,147.500228,0.006288
9095,16/10/2022,28.503759,109.258592,0.157873,151.64,7.553109,3.681029,21.083073,1.841185,1.019155,169.472864,150.780551,147.500228,0.006288
9096,17/10/2022,28.820895,111.232677,0.156977,152.50,7.624581,3.703902,21.190857,1.851739,1.024659,173.499250,153.143201,150.121000,0.006271
9097,18/10/2022,29.002505,111.069118,0.157693,152.82,7.627652,3.707873,21.218222,1.855512,1.024125,172.976958,153.541646,150.558264,0.006255


In [8]:
ponderaciones_final = cotizaciones_final[['Período']].merge(
    ponderaciones_completo, on='Período')
itcrm_final = cotizaciones_final[['Período']].merge(itcrm, on='Período')

# Canasta_t=itcr_t/cotiz_t

canastas = pd.DataFrame()
canastas['Período'] = itcrm_final.Período
lista_paises = ponderaciones_final.columns[1:]

for i in range(len(lista_paises)):
    canastas[lista_paises[i]] = itcrm_final[itcrm_final.columns[i+2]] / \
        cotizaciones_final[cotizaciones_final.columns[i+1]]
canastas


,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam
0,21/11/1997,130.059660,104.781714,40770.936982,71.409668,662.833523,753.634398,155.766407,392.283357,3735.757243,49.662325,95.871446,67.303052,35926.152319
1,22/11/1997,130.075399,104.784613,40775.552029,71.417219,663.122113,753.892907,155.730046,392.337812,3736.300050,49.667075,95.876932,67.310472,35932.909828
2,23/11/1997,130.091140,104.787513,40780.167540,71.424771,663.410828,754.151505,155.693694,392.392273,3736.842933,49.671825,95.882419,67.317892,35939.668558
3,24/11/1997,130.553885,104.791961,41398.736321,71.432324,661.396445,749.863486,155.659231,388.943674,3743.499341,49.714873,96.055695,67.808972,35848.346299
4,25/11/1997,130.569684,104.794861,41403.422384,71.439877,661.684410,750.120702,155.621015,386.738559,3744.043268,49.719628,96.061192,67.491937,35906.268925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,15/10/2022,2.994503,0.907752,519.840927,0.673146,12.909596,27.872432,4.046635,49.674459,89.778483,0.446639,0.564570,0.590434,14906.994639
9095,16/10/2022,2.989896,0.906196,519.055500,0.672028,12.888038,27.827669,4.037416,49.600548,89.649973,0.445930,0.563531,0.589450,14877.136899
9096,17/10/2022,2.985296,0.904643,518.271260,0.670912,12.866516,27.782976,4.028217,49.526748,89.521647,0.445221,0.562494,0.588467,14847.338953
9097,18/10/2022,2.980703,0.903093,517.488204,0.669798,12.845029,27.738356,4.019040,49.453057,89.393504,0.444513,0.561459,0.587487,14817.600698


In [9]:
indice = itcrm[itcrm.Período == cotizaciones.Período.iloc[0]].index-1
itcrm_rezago = itcrm[(itcrm.index >= indice[0]) & (
    itcrm.index <= itcrm.index[-2])].reset_index(drop=True)

itcrm_rezago


,Período,ITCRM,ITCRB Brasil,ITCRB Canadá,ITCRB Chile,ITCRB Estados Unidos,ITCRB México,ITCRB Uruguay,ITCRB China,ITCRB India,ITCRB Japón,ITCRB Reino Unido,ITCRB Suiza,ITCRB Zona Euro,ITCRB Vietnam,ITCRB Sudamérica*
0,20/11/1997,83.658000,117.792696,73.854316,96.633511,71.401256,80.278131,75.678549,58.510714,55.259287,29.474352,84.278556,68.481639,80.226803,59.036581,96.700000
1,21/11/1997,83.421000,117.626477,74.049237,96.464037,71.409668,80.735112,75.818635,58.497763,54.842390,29.732892,83.978991,67.723590,79.484905,58.883718,96.600000
2,22/11/1997,83.431000,117.640711,74.051286,96.474956,71.417219,80.770263,75.844642,58.484107,54.850003,29.737212,83.987024,67.727465,79.493667,58.894794,96.700000
3,23/11/1997,83.441000,117.654947,74.053335,96.485876,71.424771,80.805429,75.870658,58.470455,54.857617,29.741533,83.995057,67.731341,79.502430,58.905872,96.700000
4,24/11/1997,83.611000,117.764696,73.846895,96.541853,71.432324,80.364298,75.896683,58.457513,54.375492,29.528723,84.142423,68.497316,80.082396,58.756192,96.700000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,14/10/2022,88.204000,85.619612,99.505174,82.321377,102.430690,97.823209,102.924741,85.643856,91.739182,91.772429,75.932168,85.416298,87.370743,94.072989,90.288577
9095,15/10/2022,87.911000,85.354600,99.179749,82.068847,102.075844,97.507589,102.599232,85.315512,91.459868,91.498190,75.693273,85.126192,87.089135,93.738193,90.007560
9096,16/10/2022,87.758000,85.223278,99.009753,81.944849,101.906320,97.344756,102.434455,85.121136,91.323785,91.367218,75.572953,84.969544,86.943986,93.550442,89.867527
9097,17/10/2022,88.388000,86.038902,100.625890,81.356668,102.314062,98.101790,102.905422,85.361380,91.710610,91.729161,77.245456,86.142162,88.341319,93.101115,90.100331


In [10]:
cotizaciones_final[cotizaciones_final.columns[0+1]]

0        0.904404
1        0.904404
2        0.904404
3        0.902039
4        0.902039
          ...    
9094    28.503759
9095    28.503759
9096    28.820895
9097    29.002505
9098    29.002026
Name: Brasil, Length: 9099, dtype: float64

### Productoria

Primero calculo la original, que parte del paso para calcular el itcrm

In [11]:
productoria_original = pd.DataFrame()
productoria_original['Período'] = itcrm_final.Período

for i in range(len(lista_paises)):
    productoria_original[lista_paises[i]] = ((cotizaciones_final[cotizaciones_final.columns[i+1]]
                                    # / cotizaciones_final["Estados Unidos"]
                                     * canastas[canastas.columns[i+1]]
                                     / itcrm_rezago[itcrm_rezago.columns[i+2]]
                                     )**(ponderaciones_final[ponderaciones_final.columns[i+1]]/100))

productoria_original['productoria'] = productoria_original[productoria_original.columns[1:]
                                         ].product(axis=1)

productoria_original

,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam,productoria
0,21/11/1997,0.999555,1.000026,0.999918,1.000025,1.000112,1.000062,0.999994,0.999933,1.000246,0.999907,0.999932,0.997506,0.999999,0.997217
1,22/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
2,23/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
3,24/11/1997,1.000294,0.999972,1.000027,1.000022,0.999892,1.000011,0.999994,0.999922,0.999797,1.000046,1.000069,1.001955,0.999999,1.002001
4,25/11/1997,1.000038,1.000022,1.000005,1.000022,1.000009,1.000011,0.999993,0.999950,0.999871,0.999895,0.999989,0.998741,1.000000,0.998547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,15/10/2022,0.999215,0.999965,0.999863,0.999562,0.999922,0.999928,0.999263,0.999785,0.999953,0.999965,0.999938,0.999416,0.999893,0.996673
9095,16/10/2022,0.999610,0.999982,0.999933,0.999790,0.999960,0.999963,0.999562,0.999895,0.999977,0.999982,0.999967,0.999698,0.999940,0.998261
9096,17/10/2022,1.002414,1.000171,0.999679,1.000505,1.000188,1.000105,1.000541,1.000298,1.000062,1.000242,1.000249,1.002890,0.999856,1.007219
9097,18/10/2022,1.001201,0.999966,1.000135,1.000055,0.999969,0.999988,0.999810,1.000038,0.999969,0.999949,1.000014,1.000225,0.999868,1.001187


In [12]:
productoria_equilibrio = pd.DataFrame()
productoria_equilibrio['Período'] = itcrm_final.Período

for i in range(len(lista_paises)):
    productoria_equilibrio[lista_paises[i]] = ((cotizaciones_final[cotizaciones_final.columns[i+1]]
                                    / cotizaciones_final["Estados Unidos"]
                                     * canastas[canastas.columns[i+1]]
                                     / itcrm_rezago[itcrm_rezago.columns[i+2]])
                                    **(ponderaciones_final[ponderaciones_final.columns[i+1]]/100))

productoria_equilibrio['productoria'] = productoria_equilibrio[productoria_equilibrio.columns[1:]
                                         ].product(axis=1)

productoria_equilibrio

,Período,Brasil,Canadá,Chile,Estados Unidos,México,Uruguay,China,India,Japón,Reino Unido,Suiza,Zona Euro,Vietnam,productoria
0,21/11/1997,0.999555,1.000026,0.999918,1.000025,1.000112,1.000062,0.999994,0.999933,1.000246,0.999907,0.999932,0.997506,0.999999,0.997217
1,22/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
2,23/11/1997,1.000038,1.000000,1.000005,1.000022,1.000009,1.000011,0.999994,1.000001,1.000004,1.000002,1.000000,1.000030,1.000000,1.000117
3,24/11/1997,1.000294,0.999972,1.000027,1.000022,0.999892,1.000011,0.999994,0.999922,0.999797,1.000046,1.000069,1.001955,0.999999,1.002001
4,25/11/1997,1.000038,1.000022,1.000005,1.000022,1.000009,1.000011,0.999993,0.999950,0.999871,0.999895,0.999989,0.998741,1.000000,0.998547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9094,15/10/2022,0.280254,0.948416,0.799450,0.529979,0.885452,0.891504,0.381144,0.702024,0.923799,0.945866,0.912675,0.402717,0.860372,0.006573
9095,16/10/2022,0.280365,0.948432,0.799506,0.530100,0.885485,0.891535,0.381258,0.702101,0.923822,0.945882,0.912701,0.402831,0.860412,0.006583
9096,17/10/2022,0.280749,0.948554,0.799102,0.530100,0.885566,0.891546,0.381218,0.702104,0.923818,0.946069,0.912865,0.403704,0.860194,0.006605
9097,18/10/2022,0.280260,0.948339,0.799392,0.529721,0.885328,0.891399,0.380786,0.701818,0.923701,0.945769,0.912616,0.402478,0.860150,0.006551


In [13]:
# Tipo de cambio de equilibrio = 1/(productoria_t*ITCRM_rezagado/promedio)
prom_inicio = itcrm[itcrm.Período == '01/07/2002'].index[0]
prom_final = itcrm[itcrm.Período == '01/01/2007'].index[0]
promedio_2002_2007 = itcrm[(itcrm.index >= prom_inicio) & (
    itcrm.index < prom_final)].mean()[0]

tc_equilibrio = pd.DataFrame()
tc_equilibrio['Período'] = itcrm_final.Período

tc_equilibrio['tc_equilibrio'] = 1 / \
    (productoria_equilibrio.productoria*itcrm_rezago['ITCRM ']/promedio_2002_2007)
tc_equilibrio['tc_oficial_mayorista'] = cotizaciones_final["Estados Unidos"]
tc_equilibrio['Período'] = pd.to_datetime(
    tc_equilibrio['Período'], format='%d/%m/%Y')
tc_equilibrio['brecha'] = tc_equilibrio.tc_equilibrio / \
    tc_equilibrio.tc_oficial_mayorista-1

tc_equilibrio


C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_3260\999697203.py:4: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



,Período,tc_equilibrio,tc_oficial_mayorista,brecha
0,1997-11-21,1.928716,1.00,0.928716
1,1997-11-22,1.928586,1.00,0.928586
2,1997-11-23,1.928355,1.00,0.928355
3,1997-11-24,1.924498,1.00,0.924498
4,1997-11-25,1.927228,1.00,0.927228
...,...,...,...,...
9094,2022-10-15,277.547930,151.64,0.830308
9095,2022-10-16,278.030107,151.64,0.833488
9096,2022-10-17,277.603332,152.50,0.820350
9097,2022-10-18,277.866896,152.82,0.818263


In [14]:
idx = tc_equilibrio[tc_equilibrio.Período == '2006-01-01'].index[0]
tc_equilibrio[tc_equilibrio.index >= idx]

,Período,tc_equilibrio,tc_oficial_mayorista,brecha
2963,2006-01-01,3.003640,3.032,-0.009354
2964,2006-01-02,3.004371,3.030,-0.008458
2965,2006-01-03,2.999972,3.038,-0.012518
2966,2006-01-04,2.969472,3.039,-0.022879
2967,2006-01-05,2.972847,3.050,-0.025296
...,...,...,...,...
9094,2022-10-15,277.547930,151.640,0.830308
9095,2022-10-16,278.030107,151.640,0.833488
9096,2022-10-17,277.603332,152.500,0.820350
9097,2022-10-18,277.866896,152.820,0.818263


In [15]:
def plot_brecha(df=tc_equilibrio, anio_ini='2003', mes='01', dia='01'):
    idx = df[df.Período == f'{str(anio_ini)}-{str(mes)}-{str(dia)}'].index[0]
    df = df[df.index >= idx].reset_index(drop=True)
    x = df["Período"]
    ultima_fecha = df.Período.iloc[-1]
    ultimo_tc_equil = round(df.tc_equilibrio.iloc[-1], 2)
    ultimo_tc_may = df.tc_oficial_mayorista.iloc[-1]
    ultima_brecha = df.brecha.iloc[-1]

    # Datos
    cotizaciones_plot = make_subplots(specs=[[{"secondary_y": True}]])
    cotizaciones_plot.add_trace(go.Scatter(
        name="Tipo de cambio de equilibro", x=x, y=df["tc_equilibrio"], mode="lines"), secondary_y=False)
    cotizaciones_plot.add_trace(go.Scatter(name="Tipo de cambio oficial mayorista",
                                x=x, y=df["tc_oficial_mayorista"], mode="lines"), secondary_y=False)
    cotizaciones_plot.add_trace(go.Scatter(
        name="Brecha (der)", x=x, y=df["brecha"], mode="lines"), secondary_y=True)

    # Propiedades
    cotizaciones_plot.update_xaxes(showgrid=True)
    cotizaciones_plot.update_yaxes(
        title_text="Cotizaciones", secondary_y=False, zeroline=False)
    cotizaciones_plot.update_yaxes(
        title_text="Brecha (devaluación requerida)", tickformat=',.0%', secondary_y=True)
    cotizaciones_plot.update_layout(separators=",.", font_family="Georgia",
                                    margin ={'b': 50,'l':50,'r':15},
                                    height=600, width=900,
                                    template='none',
                                    title_text=f"Tipo de cambio nominal de equilibrio macroeconómico {str(anio_ini)}-{str(ultima_fecha.year)}<br><sup>Tipo de cambio que dejaría al ITCRM igual al promedio jul/02-dic/06: {str(round(promedio_2002_2007,1)).replace('.',',')}",
                                    title_font=dict(size=20),
                                    legend=dict(yanchor="top", y=1.05, xanchor="left", x=0.15, orientation='h'))

    # Flechas
    cotizaciones_plot.add_annotation(x=ultima_fecha, y=ultimo_tc_equil, align="left",
                                     text='$'+str(ultimo_tc_equil).replace('.', ','), showarrow=True, arrowhead=1)
    cotizaciones_plot.add_annotation(ax=0, ay=35, x=ultima_fecha, y=ultimo_tc_may, align="left",
                                     text='$'+str(ultimo_tc_may).replace('.', ','), showarrow=True, arrowhead=1)
    
    #Regímnes cambiarios
    anio_ini=int(anio_ini)
    if anio_ini<2007: cotizaciones_plot.add_vline(x="2006-12-12", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2012: cotizaciones_plot.add_vline(x="2011-10-28", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2016:cotizaciones_plot.add_vline(x="2015-12-17", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2019:cotizaciones_plot.add_vline(x="2018-09-26", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2020:cotizaciones_plot.add_vline(x="2019-09-01", line_width=1, line_dash="dash", line_color="Black",opacity=0.5)
    if anio_ini<2004:cotizaciones_plot.add_annotation(showarrow=False, text=f'TC competitivo y estable<br>"Crawling peg"',
        font=dict(size=13), font_family="georgia",x='2004-12-1', y=75)
    if anio_ini<2009:cotizaciones_plot.add_annotation(showarrow=False, text=f'Apreciación cambiaria<br>Inicio intervención INDEC',
        font=dict(size=13), font_family="georgia",x='2009-7-1', y=200)
    if anio_ini<2013:cotizaciones_plot.add_annotation(showarrow=False, text=f'CEPO cambiario<br>INDEC intervenido',
        font=dict(size=13), font_family="georgia",x='2013-11-1', y=250)
    if anio_ini<2017:cotizaciones_plot.add_annotation(showarrow=False, text=f'TC flotante<br>"Flotación sucia"',
        font=dict(size=13), font_family="georgia",x='2017-5-1', y=75)
    if anio_ini<2019:cotizaciones_plot.add_annotation(showarrow=False, text=f"Bandas cambiarias",
        font=dict(size=13), font_family="georgia",x='2019-03-1', y=175,textangle=90)
    if anio_ini<2021:cotizaciones_plot.add_annotation(showarrow=False, text=f'Apreciación cambiaria<br>Con CEPO',
        font=dict(size=13), font_family="georgia",x='2021-9-1', y=50) 
    
    # Nota al pie
    note = 'Fuente: BCRA'
    cotizaciones_plot.add_annotation(showarrow=False, text=note, font=dict(size=12), xref='paper', x=0.1, yref='paper', y=-0.1,
                                     xanchor='right', yanchor='auto', xshift=0, yshift=0,)

    # Marca de agua
    cotizaciones_plot.add_annotation(showarrow=False, text='@MartinBasualdo0', font=dict(size=16), font_family="arial", opacity=0.4,
                                     xref='paper', yref='paper', x=0.5, y=0.5)

    cotizaciones_plot.add_annotation(showarrow=False, text=f"Devaluación requerida al {ultima_fecha.strftime('%d/%m/%y')}:<br>{'{:.1%}'.format(ultima_brecha).replace('.',',')}", font=dict(size=14), font_family="georgia",
                                     # xref='paper', yref='paper',
                                     x='2019-05-30', y=250)
    return cotizaciones_plot


plot_brecha(anio_ini=2003)


In [16]:
writer = pd.ExcelWriter(f'../output/ITCRM historico.xlsx', engine='xlsxwriter')
itcrm.to_excel(writer, sheet_name='ITCRM', index=False)
ponderaciones_completo.to_excel(
    writer, sheet_name='ponderaciones', index=False)
cotizaciones_final.to_excel(writer, sheet_name='cotizaciones', index=False)
canastas.to_excel(writer, sheet_name='canastas', index=False)
tc_equilibrio.to_excel(writer, sheet_name='tc_equilibrio', index=False)
writer.save()


In [17]:
plot_brecha(anio_ini=2003).write_html('../output/grafico_brecha.html')
indice_monedas(anio=2007).write_html('../output/indice_monedas.html')
plot_tasa_deva(anio=2007).write_html('../output/grafico_devaluacion_socios.html')